# Wrangle Exercises

In [1]:
import pandas as pd
import numpy as np

import pyspark
spark = pyspark.sql.SparkSession.builder.getOrCreate()
from pyspark.sql.functions import *

#  This exercises uses the case.csv, dept.csv, and source.csv files from the san antonio 311 call dataset.

## 1. Read the case, department, and source data into their own spark dataframes.

In [2]:
#Create the source df
source_df = spark.read.csv("source.csv", sep=",", header=True, inferSchema=True)

#Create the department df
dept_df = spark.read.csv("dept.csv", sep=",", header=True, inferSchema=True)

#Create the case df
case_df = spark.read.csv("case.csv", header=True, inferSchema=True)

In [49]:
print("--- case ---")
case_df.show(1, vertical=True)
print("--- dept ---")
dept_df.show()
print("--- source ---")
source_df.show()

--- case ---
-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 2018-01-01 00:42:00  
 case_closed_date     | 2018-01-01 00:42:00  
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
 case_due_date        | 2018-01-01 00:42:00  
 year                 | 2018                 
only showing top 1 row

--- dept ---
+--------------------+--------------------+----------------------+-------------------+
|       dept_division|           dept_name|standardized_dept_name|dept_subject_to

## 2. Let's see how writing to the local disk works in spark:

    - Write the code necessary to store the source data in both csv and json format, store these as sources_csv and sources_json
    

In [4]:
# write to json
source_df.write.json("source_json", mode="overwrite")

In [5]:
#Write to csv
source_df.write.csv("source_csv", mode="overwrite")

    - Inspect your folder structure. What do you notice?
    

    - There are folders for each json and csv. Within each file folder is a success document showing that the write was a success and a document with the data

## 3. Inspect the data in your dataframes. Are the data types appropriate? Write the code necessary to cast the values to the appropriate types.

In [6]:
#Check the dtypes for source_df
source_df

#takeaways: source_id is a string and can stay that way for now

DataFrame[source_id: string, source_username: string]

In [7]:
#Check the dtypes for dept
dept_df

#takeaways:
# All good except for dept_subject_to_SLA could be converted to abool

DataFrame[dept_division: string, dept_name: string, standardized_dept_name: string, dept_subject_to_SLA: string]

In [8]:
#check the dtypes for case_df
case_df

#takeaways:
#case_id --> string
#case_opened_date, case_closed_date, SLA_due_date --> datetime
#case_late --> abool
#SLA_days --> int
#council_district --> string

DataFrame[case_id: int, case_opened_date: string, case_closed_date: string, SLA_due_date: string, case_late: string, num_days_late: double, case_closed: string, dept_division: string, service_request_type: string, SLA_days: double, case_status: string, source_id: string, request_address: string, council_district: int]

In [9]:
case_df.show(1, vertical=True)

-RECORD 0------------------------------------
 case_id              | 1014127332           
 case_opened_date     | 1/1/18 0:42          
 case_closed_date     | 1/1/18 12:29         
 SLA_due_date         | 9/26/20 0:42         
 case_late            | NO                   
 num_days_late        | -998.5087616000001   
 case_closed          | YES                  
 dept_division        | Field Operations     
 service_request_type | Stray Animal         
 SLA_days             | 999.0                
 case_status          | Closed               
 source_id            | svcCRMLS             
 request_address      | 2315  EL PASO ST,... 
 council_district     | 5                    
only showing top 1 row



In [19]:
#case_id --> string
case_df.select(case_df.case_id.cast("string"))

#case_opened_date, case_closed_date, SLA_due_date --> datetime

fmt = "M/d/yy H:mm"
case_df = (
    case_df.withColumn("case_opened_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_closed_date", to_timestamp("case_opened_date", fmt))
    .withColumn("case_due_date", to_timestamp("case_opened_date", fmt))
)

#case_late --> abool


#SLA_days --> int
#council_district --> string

---

## 1. How old is the latest (in terms of days past SLA) currently open issue? How long has the oldest (in terms of days since opened) currently opened issue been open?

In [21]:
#How old is the latest currently open issue?
case_df.where((case_df.case_closed == 'NO') & (case_df.num_days_late > 0)).select(min(case_df.num_days_late)).show()

+------------------+
|min(num_days_late)|
+------------------+
|       0.060810185|
+------------------+



In [22]:
#How long has the oldest (in terms of days since opened) currently opened issue been open?
case_df.where((case_df.case_closed == 'NO') & (case_df.num_days_late > 0)).select(max(case_df.num_days_late)).show()

+------------------+
|max(num_days_late)|
+------------------+
|       348.6458333|
+------------------+



## 2. How many Stray Animal cases are there?

In [50]:
# Need to groupby case type
(
    case_df.groupBy('service_request_type').count()
    .where(case_df.service_request_type == 'Stray Animal').show()
)

+--------------------+-----+
|service_request_type|count|
+--------------------+-----+
|        Stray Animal|26760|
+--------------------+-----+



## 3. How many service requests that are assigned to the Field Operations department (dept_division) are not classified as "Officer Standby" request type (service_request_type)?

In [24]:
(
    case_df.select('service_request_type')
    .where(case_df.dept_division == 'Field Operations')
    .where(case_df.service_request_type != 'Officer Standby').count()
)

113902

## 4. Convert the council_district column to a string column.

In [25]:
case_df = case_df.withColumn("council_district", col("council_district").cast("string"))

In [26]:
case_df.printSchema()

root
 |-- case_id: integer (nullable = true)
 |-- case_opened_date: timestamp (nullable = true)
 |-- case_closed_date: timestamp (nullable = true)
 |-- SLA_due_date: string (nullable = true)
 |-- case_late: string (nullable = true)
 |-- num_days_late: double (nullable = true)
 |-- case_closed: string (nullable = true)
 |-- dept_division: string (nullable = true)
 |-- service_request_type: string (nullable = true)
 |-- SLA_days: double (nullable = true)
 |-- case_status: string (nullable = true)
 |-- source_id: string (nullable = true)
 |-- request_address: string (nullable = true)
 |-- council_district: string (nullable = true)
 |-- case_due_date: timestamp (nullable = true)



## 5. Extract the year from the case_closed_date column.

In [35]:
case_df = case_df.withColumn('year', year("case_closed_date"))

In [36]:
case_df.select(case_df.year).show(1)

+----+
|year|
+----+
|2018|
+----+
only showing top 1 row



## 6. Convert num_days_late from days to hours in new columns num_hours_late.

In [38]:
case_df.withColumn('num_hours_late', case_df.num_days_late / 24).show(5)

+----------+-------------------+-------------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+
|   case_id|   case_opened_date|   case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|   dept_division|service_request_type|   SLA_days|case_status|source_id|     request_address|council_district|      case_due_date|year|      num_hours_late|
+----------+-------------------+-------------------+------------+---------+-------------------+-----------+----------------+--------------------+-----------+-----------+---------+--------------------+----------------+-------------------+----+--------------------+
|1014127332|2018-01-01 00:42:00|2018-01-01 00:42:00|9/26/20 0:42|       NO| -998.5087616000001|        YES|Field Operations|        Stray Animal|      999.0|     Closed| svcCRMLS|2315  EL PASO ST,...|        

## 7. Join the case data with the source and department data.

In [39]:
#first I'll join the case and source dataframes
case_source = case_df.join(source_df, on='source_id', how='left')

#Then join the dept df to the case_source df
case_source_dept = case_source.join(dept_df, on='dept_division', how='left')


+----------------+---------+----------+-------------------+-------------------+------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+-------------------+----+---------------+--------------------+----------------------+-------------------+
|   dept_division|source_id|   case_id|   case_opened_date|   case_closed_date|SLA_due_date|case_late|      num_days_late|case_closed|service_request_type|   SLA_days|case_status|     request_address|council_district|      case_due_date|year|source_username|           dept_name|standardized_dept_name|dept_subject_to_SLA|
+----------------+---------+----------+-------------------+-------------------+------------+---------+-------------------+-----------+--------------------+-----------+-----------+--------------------+----------------+-------------------+----+---------------+--------------------+----------------------+-------------------+
|Field Operations| svcCRMLS|101

In [40]:
#Check the join
case_source_dept.show(1,  vertical=True)

-RECORD 0--------------------------------------
 dept_division          | Field Operations     
 source_id              | svcCRMLS             
 case_id                | 1014127332           
 case_opened_date       | 2018-01-01 00:42:00  
 case_closed_date       | 2018-01-01 00:42:00  
 SLA_due_date           | 9/26/20 0:42         
 case_late              | NO                   
 num_days_late          | -998.5087616000001   
 case_closed            | YES                  
 service_request_type   | Stray Animal         
 SLA_days               | 999.0                
 case_status            | Closed               
 request_address        | 2315  EL PASO ST,... 
 council_district       | 5                    
 case_due_date          | 2018-01-01 00:42:00  
 year                   | 2018                 
 source_username        | svcCRMLS             
 dept_name              | Animal Care Services 
 standardized_dept_name | Animal Care Services 
 dept_subject_to_SLA    | YES           

## 8. Are there any cases that do not have a request source?

In [47]:
case_source_dept.rollup(case_source_dept.source_username).count().sort(case_source_dept.source_username).show()

AnalysisException: Column source_username#1255 are ambiguous. It's probably because you joined several Datasets together, and some of these Datasets are the same. This column points to one of the Datasets but Spark is unable to figure out which one. Please alias the Datasets with different names via `Dataset.as` before joining them, and specify the column using qualified name, e.g. `df.as("a").join(df.as("b"), $"a.id" > $"b.id")`. You can also set spark.sql.analyzer.failAmbiguousSelfJoin to false to disable this check.

## 9. What are the top 10 service request types in terms of number of requests?

In [52]:
(
    case_source_dept.select('service_request_type')
    .groupBy('service_request_type').count()
    .sort(desc('count')).show(5,truncate=False)
)

+--------------------------+-----+
|service_request_type      |count|
+--------------------------+-----+
|No Pickup                 |89210|
|Overgrown Yard/Trash      |66403|
|Bandit Signs              |32968|
|Damaged Cart              |31163|
|Front Or Side Yard Parking|28920|
+--------------------------+-----+
only showing top 5 rows



## 10. What are the top 10 service request types in terms of average days late?

In [55]:
(
    case_source_dept.groupBy('service_request_type')
    .agg(round(mean(case_source_dept.num_days_late),2).alias('avg_days_late'))
    .sort(desc('avg_days_late')).show(10,truncate=False)
)

+--------------------------------------+-------------+
|service_request_type                  |avg_days_late|
+--------------------------------------+-------------+
|Zoning: Junk Yards                    |175.96       |
|Labeling for Used Mattress            |162.43       |
|Record Keeping of Used Mattresses     |154.0        |
|Signage Requied for Sale of Used Mattr|151.64       |
|Storage of Used Mattress              |142.11       |
|Zoning: Recycle Yard                  |135.93       |
|Donation Container Enforcement        |131.76       |
|License Requied Used Mattress Sales   |128.8        |
|Traffic Signal Graffiti               |101.8        |
|Complaint                             |72.87        |
+--------------------------------------+-------------+
only showing top 10 rows



## 11. Does number of days late depend on department?

In [56]:
(
    case_source_dept.groupBy('dept_name').agg(round(mean(case_source_dept.num_days_late),2).alias('avg_days_late'))
    .sort(desc('avg_days_late')).show(truncate=False)
)

+-------------------------+-------------+
|dept_name                |avg_days_late|
+-------------------------+-------------+
|null                     |135.93       |
|Customer Service         |59.74        |
|Development Services     |13.43        |
|Solid Waste Management   |-2.2         |
|Metro Health             |-4.91        |
|Parks and Recreation     |-5.25        |
|Trans & Cap Improvements |-20.61       |
|Code Enforcement Services|-38.7        |
|Animal Care Services     |-226.52      |
|City Council             |null         |
+-------------------------+-------------+



## 12. How do number of days late depend on department and request type?

In [57]:
(
    case_source_dept.groupBy('dept_name','service_request_type').agg(round(mean(case_source_dept.num_days_late),2).alias('avg_days_late'))
    .sort(asc('dept_name'),desc('service_request_type'),desc('avg_days_late')).show(truncate=False)

)

+-------------------------+--------------------------------------+-------------+
|dept_name                |service_request_type                  |avg_days_late|
+-------------------------+--------------------------------------+-------------+
|null                     |Zoning: Recycle Yard                  |135.93       |
|Animal Care Services     |Trapped/Confined Animal               |0.25         |
|Animal Care Services     |Stray Animal                          |-998.81      |
|Animal Care Services     |Spay/Neuter Request Response          |-6.98        |
|Animal Care Services     |Public Nuisance(Own Animal)           |-2.2         |
|Animal Care Services     |Officer Standby                       |-0.07        |
|Animal Care Services     |Injured Animal(Critical)              |-0.07        |
|Animal Care Services     |City Council Animal Request           |-1.55        |
|Animal Care Services     |Animal Permits Request                |22.2         |
|Animal Care Services     |A